In [1]:
sc.install_pypi_package("h5py")
sc.install_pypi_package("s3fs")
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1617155652825_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Created wheel for aiobotocore: filename=aiobotocore-1.2.2-py3-none-any.whl size=45730 sha256=06768e462e4284c73878e587bfe8a65a5342904052267f5d9b60d34626960870
  Stored in directory: /mnt/var/lib/livy/.cache/pip/wheels/37/f3/76/dfc2d32494696a7e4710b2f57d9d15212226d19c42dc395865
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3160 sha256=bac602afb0a1af699037e160c4cb1d107abe049d1e3638dae7105a75b5519c22
  Stored in directory: /mnt/var/lib/livy/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built aiobotocore idna-ssl

  Attempting uninstall: botocore
    Found existing installation: botocore 1.19.52
    Uninstalling botocore-1.19.52:
      Successfully uninstalled botocore-1.19.52



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 1.2.2 requires botocore<1.19.53,>=1.19.52, but you 

In [1]:
# import s3fs
# import io
# import h5py
# import boto
import numpy as np

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1617185507602_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
parDF1=spark.read.parquet("s3://17700project/song_features.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
parDF1.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[analysis_sample_rate: bigint, audio_md5: string, danceability: double, duration: double, end_of_fade_in: double, energy: double, idx_bars_confidence: bigint, idx_bars_start: bigint, idx_beats_confidence: bigint, idx_beats_start: bigint, idx_sections_confidence: bigint, idx_sections_start: bigint, idx_segments_confidence: bigint, idx_segments_loudness_max: bigint, idx_segments_loudness_max_time: bigint, idx_segments_loudness_start: bigint, idx_segments_pitches: bigint, idx_segments_start: bigint, idx_segments_timbre: bigint, idx_tatums_confidence: bigint, idx_tatums_start: bigint, key: bigint, key_confidence: double, loudness: double, mode: bigint, mode_confidence: double, start_of_fade_out: double, tempo: double, time_signature: bigint, time_signature_confidence: double, track_id: string, analyzer_version: string, artist_7digitalid: bigint, artist_familiarity: double, artist_hotttnesss: double, artist_id: string, artist_latitude: double, artist_location: string, artist_longit

In [4]:
parDF1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1000000

Reference Repo : https://github.com/RohanBhirangi/Million-Song-Dataset-Analysis

##Feature Engineering :

We only considered the following features for our use : TrackID, Artist name, Artist location, User play count, Genre, Lyrics, Mode (major/minor), Song Duration, Song Loudness, Song hotness, Artist hotness, Tempo (in bpm), Time Signature and Time Signature Confidence. As some of these features were available across different dataframes, they had to be joined together to a single dataframe. Additionally, rows with a time signature confidence threshold less than 0.5 had to be dropped because they were filled with garbage values which was not accurate. Another feature “Speed” had to be calculated from each tracks’ tempo and time signature by calculating the number of beats per measure rather than the number of beats per time (tempo). This is done to normalize how “fast” a song sounds to human ears. Natural Language Processing techniques like standardizing (removing non-english and other irrelevant characters) and stop word removal (remove frequent meaningless words like ‘I’, ‘the’, ‘a’, etc.) were also done on the lyrics to preserve only the words with actual meaning.

## Song Recommender System :
We wanted to build a system to recommend new songs to a user based on his tastes and listening history. The traditional approach of collaborative filtering was discarded because we do not have a metric of user ratings (only play counts) and song features would not be used in this case. To solve this problem, we used a clustering technique (k-means) in order to group songs together, which would incorporate song features as well as reduce the search space while recommending new songs. The model was trained on the following features : mode, duration, loudness, genre, speed and hotness. Each of the features had to be cleaned to remove false outliers and then min-max normalized (scaled) to a value between 1 and 5. The optimal number of clusters was found by calculating the sum of square errors within each cluster for different values of k. The optimal value was found to be 7. We now look at the user history and retrieve his/her 5 most played songs. For each of these, we search within the corresponding cluster for the 3 most similar songs. The similarity measure used is the Manhattan distance between scaled features. The 15 obtained results are ranked based on the user’s favorite genres and similarity scores provided for each. Limitations for the above approach include that it is not suitable for dynamic data (retraining the model) and that there is no objective way to measure performance.


In [5]:
from pyspark.ml.linalg import DenseVector#Shayeree's addition
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import BisectingKMeans
from pyspark.sql.functions import lit
from  pyspark.sql.functions import abs
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.sql import functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
msdf2 =parDF1.withColumn("time_signature", F.when(parDF1.time_signature_confidence<0.5, 4).otherwise(parDF1.time_signature))
msdf2.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+------------+---------+--------------+------+-------------------+--------------+--------------------+---------------+-----------------------+------------------+-----------------------+-------------------------+------------------------------+---------------------------+--------------------+------------------+-------------------+---------------------+----------------+---+--------------+--------+----+---------------+-----------------+------+--------------+-------------------------+--------------------+----------------+-----------------+------------------+------------------+--------------------+---------------+---------------+----------------+--------------------+-----------+---------------+-----+----------------+-------------------+--------------------+------------------+------------------+--------------------+----------+----------------+-----------------+----+
|analysis_sample_rate|           audio_md5|danceability| duration|end_of_fade_in|energ

In [7]:
selected_numerical_features = ['duration', 'end_of_fade_in', 'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence',\
                               'start_of_fade_out', 'tempo', 'time_signature', 'time_signature_confidence']
msdf3 = msdf2.select(msdf2.duration, msdf2.end_of_fade_in, msdf2.key,msdf2.key_confidence,msdf2.mode, msdf2.mode_confidence,msdf2.loudness,msdf2.start_of_fade_out, msdf2.time_signature_confidence,msdf2.artist_hotttnesss,msdf2.time_signature.alias("speed"), msdf2.song_id)
#msdf4 = msdf3.na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
msdf3.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------+---+--------------+----+---------------+--------+-----------------+-------------------------+------------------+-----+--------------------+
| duration|end_of_fade_in|key|key_confidence|mode|mode_confidence|loudness|start_of_fade_out|time_signature_confidence| artist_hotttnesss|speed|             song_id|
+---------+--------------+---+--------------+----+---------------+--------+-----------------+-------------------------+------------------+-----+--------------------+
|209.08363|           0.0|  5|         0.614|   0|           0.61|  -4.055|          206.843|                    0.799|0.5865831757223228|    4|b'SOMLEUP12A6D4F8...|
+---------+--------------+---+--------------+----+---------------+--------+-----------------+-------------------------+------------------+-----+--------------------+
only showing top 1 row

In [9]:
msdf4 = msdf3.na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
msdf4.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------+---+--------------+----+---------------+--------+-----------------+-------------------------+-------------------+-----+--------------------+
| duration|end_of_fade_in|key|key_confidence|mode|mode_confidence|loudness|start_of_fade_out|time_signature_confidence|  artist_hotttnesss|speed|             song_id|
+---------+--------------+---+--------------+----+---------------+--------+-----------------+-------------------------+-------------------+-----+--------------------+
|209.08363|           0.0|  5|         0.614|   0|           0.61|  -4.055|          206.843|                    0.799| 0.5865831757223228|    4|b'SOMLEUP12A6D4F8...|
|321.20118|         0.235|  5|         0.657|   0|          0.763| -12.303|          310.393|                      0.0| 0.4150033040402988|    4|b'SOMUXIE12A58A76...|
|164.44036|         0.233|  4|           0.0|   1|            0.0| -10.726|          156.177|                    0.518|0.38672538144513174|    4|b'SOLUWQY12A8C13E...

In [11]:
#With feature scaling
scaledRange = lit(4)
scaledMin = lit(1)

duration_stats = (msdf4.selectExpr("min(duration)","max(duration)")).collect()
dMin = duration_stats[0][0]
dMax = duration_stats[0][1]
msdf4 = (msdf4.withColumn("duration",(msdf4.duration - dMin)/(dMax-dMin)))
msdf4= msdf4.withColumn("duration",scaledRange*msdf4.duration + scaledMin)
msdf4.select("duration").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|         duration|
+-----------------+
|1.275187656570644|
|1.422973620528607|
+-----------------+
only showing top 2 rows

In [12]:
loudness_stats = (msdf4.selectExpr("min(loudness)","max(loudness)")).collect()
lMin = loudness_stats[0][0]
lMax = loudness_stats[0][1]
msdf4= (msdf4.withColumn("loudness",(msdf4.loudness - lMin)/(lMax-lMin)))
msdf4 = msdf4.withColumn("loudness",scaledRange*msdf4.loudness + scaledMin)
msdf4.select("loudness").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|         loudness|
+-----------------+
|4.464093701996928|
|3.936187916026626|
+-----------------+
only showing top 2 rows

In [13]:
artist_hotttnesss_stats = (msdf4.selectExpr("min(artist_hotttnesss)","max(artist_hotttnesss)")).collect()
aMin = artist_hotttnesss_stats[0][0]
aMax = artist_hotttnesss_stats[0][1]
msdf4 = (msdf4.withColumn("artist_hotttnesss",(msdf4.artist_hotttnesss - aMin)/(aMax-aMin)))
msdf4 = msdf4.withColumn("artist_hotttnesss",scaledRange*msdf4.artist_hotttnesss + scaledMin)
msdf4.select("artist_hotttnesss").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|artist_hotttnesss|
+-----------------+
| 3.16750777012984|
|2.533495884925825|
+-----------------+
only showing top 2 rows

In [14]:
speed_stats = (msdf4.selectExpr("min(speed)","max(speed)")).collect()
sMin = speed_stats[0][0]
sMax = speed_stats[0][1]
msdf4 = (msdf4.withColumn("speed",(msdf4.speed - sMin)/(sMax-sMin)))
msdf4 = msdf4.withColumn("speed",scaledRange*msdf4.speed + scaledMin)
msdf4.select("speed").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+
|speed|
+-----+
|  3.0|
|  3.0|
+-----+
only showing top 2 rows

In [15]:
fade_stats = (msdf4.selectExpr("min(end_of_fade_in)","max(end_of_fade_in)")).collect()
fMin = fade_stats[0][0]
fMax = fade_stats[0][1]
msdf4 = (msdf4.withColumn("end_of_fade_in",(msdf4.end_of_fade_in - fMin)/(fMax-fMin)))
msdf4 = msdf4.withColumn("end_of_fade_in",scaledRange*msdf4.end_of_fade_in + scaledMin)
msdf4.select("end_of_fade_in").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|   end_of_fade_in|
+-----------------+
|              1.0|
|1.000807601072563|
+-----------------+
only showing top 2 rows

In [16]:
key_stats = (msdf4.selectExpr("min(key)","max(key)")).collect()
kMin = key_stats[0][0]
kMax = key_stats[0][1]
msdf4 = (msdf4.withColumn("key",(msdf4.key - kMin)/(kMax-kMin)))
msdf4 = msdf4.withColumn("key",scaledRange*msdf4.key + scaledMin)
msdf4.select("key").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|               key|
+------------------+
|2.8181818181818183|
|2.8181818181818183|
+------------------+
only showing top 2 rows

In [17]:
key_conf_stats = (msdf4.selectExpr("min(key_confidence)","max(key_confidence)")).collect()
kcMin = key_conf_stats[0][0]
kcMax = key_conf_stats[0][1]
msdf4 = (msdf4.withColumn("key_confidence",(msdf4.key_confidence - kcMin)/(kcMax-kcMin)))
msdf4 = msdf4.withColumn("key_confidence",scaledRange*msdf4.key_confidence + scaledMin)
msdf4.select("key_confidence").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|key_confidence|
+--------------+
|         3.456|
|         3.628|
+--------------+
only showing top 2 rows

In [18]:
m_stats = (msdf4.selectExpr("min(mode)","max(mode)")).collect()
mMin = m_stats[0][0]
mMax = m_stats[0][1]
msdf4 = (msdf4.withColumn("mode",(msdf4.mode - mMin)/(mMax-mMin)))
msdf4 = msdf4.withColumn("mode",scaledRange*msdf4.mode + scaledMin)
msdf4.select("mode").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|mode|
+----+
| 1.0|
| 1.0|
+----+
only showing top 2 rows

In [19]:
mc_stats = (msdf4.selectExpr("min(mode_confidence)","max(mode_confidence)")).collect()
mcMin = mc_stats[0][0]
mcMax = mc_stats[0][1]
msdf4 = (msdf4.withColumn("mode_confidence",(msdf4.mode - mcMin)/(mcMax-mcMin)))
msdf4 = msdf4.withColumn("mode_confidence",scaledRange*msdf4.mode_confidence + scaledMin)
msdf4.select("mode_confidence").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|mode_confidence|
+---------------+
|            5.0|
|            5.0|
+---------------+
only showing top 2 rows

In [20]:
s_stats = (msdf4.selectExpr("min(start_of_fade_out)","max(start_of_fade_out)")).collect()
sMin = s_stats[0][0]
sMax = s_stats[0][1]
msdf4 = (msdf4.withColumn("start_of_fade_out",(msdf4.start_of_fade_out - sMin)/(sMax-sMin)))
msdf4 = msdf4.withColumn("start_of_fade_out",scaledRange*msdf4.start_of_fade_out + scaledMin)
msdf4.select("start_of_fade_out").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
| start_of_fade_out|
+------------------+
| 1.272619062940446|
|1.4093047936695564|
+------------------+
only showing top 2 rows

In [21]:
ts_stats = (msdf4.selectExpr("min(time_signature_confidence)","max(time_signature_confidence)")).collect()
tsMin = ts_stats[0][0]
tsMax = ts_stats[0][1]
msdf4 = (msdf4.withColumn("time_signature_confidence",(msdf4.time_signature_confidence - tsMin)/(tsMax-tsMin)))
msdf4 = msdf4.withColumn("time_signature_confidence",scaledRange*msdf4.time_signature_confidence + scaledMin)
msdf4.select("time_signature_confidence").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+
|time_signature_confidence|
+-------------------------+
|                    4.196|
|                      1.0|
+-------------------------+
only showing top 2 rows

In [22]:
msdf4.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------+------------------+--------------+----+---------------+-----------------+-----------------+-------------------------+-----------------+-----+--------------------+
|         duration|end_of_fade_in|               key|key_confidence|mode|mode_confidence|         loudness|start_of_fade_out|time_signature_confidence|artist_hotttnesss|speed|             song_id|
+-----------------+--------------+------------------+--------------+----+---------------+-----------------+-----------------+-------------------------+-----------------+-----+--------------------+
|1.275187656570644|           1.0|2.8181818181818183|         3.456| 1.0|            5.0|4.464093701996928|1.272619062940446|                    4.196| 3.16750777012984|  3.0|b'SOMLEUP12A6D4F8...|
+-----------------+--------------+------------------+--------------+----+---------------+-----------------+-----------------+-------------------------+-----------------+-----+--------------------+
only showing to

In [23]:
allDF=msdf4

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
allDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- duration: double (nullable = true)
 |-- end_of_fade_in: double (nullable = true)
 |-- key: double (nullable = true)
 |-- key_confidence: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- mode_confidence: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- start_of_fade_out: double (nullable = true)
 |-- time_signature_confidence: double (nullable = true)
 |-- artist_hotttnesss: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- song_id: string (nullable = true)

In [25]:
rowsdf = allDF.select(allDF.duration.cast('double'),allDF.end_of_fade_in.cast('double'), allDF.key.cast('double'), allDF.key_confidence.cast('double'), allDF.mode.cast('double'),allDF.mode_confidence.cast('double'),allDF.loudness.cast('double'),allDF.start_of_fade_out.cast('double'),allDF.time_signature_confidence.cast('double'),allDF.artist_hotttnesss.cast('double'),allDF.speed.cast('double'),allDF.song_id.cast('string'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
selected_numerical_features = ['duration', 'end_of_fade_in', 'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence',
                               'start_of_fade_out',  'time_signature_confidence','artist_hotttnesss','speed']


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
from pyspark.ml.feature import VectorAssembler

assemble=VectorAssembler(inputCols=selected_numerical_features, outputCol='features')
assembled_data=assemble.transform(rowsdf).select('song_id', 'features')
assembled_data.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|             song_id|            features|
+--------------------+--------------------+
|b'SOMLEUP12A6D4F8...|[1.27518765657064...|
|b'SOMUXIE12A58A76...|[1.42297362052860...|
|b'SOLUWQY12A8C13E...|[1.21634184080687...|
|b'SOZPMVD12AB0186...|[1.22133461636111...|
|b'SOVGENV12A8C139...|[1.23934301692848...|
|b'SOVSZTZ12A8C144...|[1.30555746584306...|
|b'SOGDSKD12AB0183...|[1.51842159441070...|
|b'SOZYAYU12A8C136...|[1.25521658071636...|
|b'SOFWYQI12AB0188...|[1.21768472944795...|
|b'SOPOZXJ12AB0187...|[1.33826870304981...|
+--------------------+--------------------+
only showing top 10 rows

In [28]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import time
import numpy as np

cost = np.zeros(20)
strt_time = time.time()
for k in range(2,40,2):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(assembled_data.sample(False,0.1, seed=42))
    cost[k//2] = model.computeCost(assembled_data) 
end_time = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
strt_time - end_time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-91.10122871398926

In [30]:
cost

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([      0.        , 5669644.30972973, 3916219.41288138,
       3488607.3207795 , 2579572.12034534, 2252939.43653187,
       2043392.63231072, 1960372.68683046, 1792587.27379728,
       1692023.96709863, 1590446.52178527, 1527659.90591236,
       1506964.97562036, 1447819.56982767, 1394033.40863645,
       1343289.47215278, 1317561.1853925 , 1274928.40902827,
       1267359.34964487, 1209823.46249561])

In [31]:
k = 60
kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
model = kmeans.fit(assembled_data)
centers = model.clusterCenters()

print("Cluster Centers: ")
for center in centers:
    print(center)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cluster Centers: 
[1.3569075  1.00263622 2.92559414 2.92456989 4.12581643 1.
 5.         1.34501762 4.82460273 2.43523815 2.97927542]
[ 1.28836005  1.00247766  3.1732665   4.22935859  4.04158278  5.
 21.          1.2774056   1.11771414  2.37405019  3.        ]
[1.36479453 1.00314037 2.8438561  2.51266902 4.06404979 1.
 5.         1.35277172 3.73521494 2.41629153 3.12849281]
[ 1.31481003  1.00287765  1.41679686  1.43403164  4.06177383  5.
 21.          1.30419563  3.01432703  2.39680767  2.99492486]
[ 1.31949293  1.00276834  1.42243066  1.34245591  4.06422634  5.
 21.          1.3085288   4.66886387  2.40414252  2.95818097]
[ 1.32177279  1.00279585  3.22199209  1.36417467  4.08189104  5.
 21.          1.31082219  3.16324446  2.40305477  2.9732827 ]
[ 1.32178651  1.00265902  3.23039468  4.2520005   4.06603287  5.
 21.          1.30995448  2.54061979  2.38415462  2.99970594]
[1.34244349 1.00287033 4.61137743 3.35116537 4.11367327 1.
 5.         1.33145649 1.22038595 2.38976201 3.        ]

In [32]:
transformed = model.transform(assembled_data).select('song_id', 'prediction')
rows = transformed.collect()

df_pred = sqlContext.createDataFrame(rows)
df_pred.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|             song_id|prediction|
+--------------------+----------+
|b'SOMLEUP12A6D4F8...|         0|
|b'SOMUXIE12A58A76...|        49|
|b'SOLUWQY12A8C13E...|         5|
|b'SOZPMVD12AB0186...|        37|
|b'SOVGENV12A8C139...|        29|
|b'SOVSZTZ12A8C144...|        45|
|b'SOGDSKD12AB0183...|        19|
|b'SOZYAYU12A8C136...|        37|
|b'SOFWYQI12AB0188...|         4|
|b'SOPOZXJ12AB0187...|         8|
|b'SOTBWME12A8C145...|        31|
|b'SOTCMPH12A58A7C...|        32|
|b'SOAJOOF12A67AD8...|         0|
|b'SORUITS12AB0183...|        10|
|b'SOMCOIX12A8C139...|         4|
|b'SOWAJNU12A8C135...|        56|
|b'SOLHSWR12A8C142...|        57|
|b'SOOUUAP12A8C142...|         3|
|b'SORIFGT12A8C135...|        31|
|b'SOEHECO12A58A79...|        29|
+--------------------+----------+
only showing top 20 rows

In [33]:

def join(rowsdf, df_pred):
    df = df_pred.join(rowsdf, rowsdf.song_id == df_pred.song_id)
    repeated_columns = [c for c in rowsdf.columns if c in df_pred.columns]
    for col in repeated_columns:
        df = df.drop(rowsdf[col])
    return df
df_final=join(rowsdf, df_pred)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_final.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+------------------+--------------+---+--------------+----+---------------+----------------+-----------------+-------------------------+------------------+-----+
|             song_id|prediction|          duration|end_of_fade_in|key|key_confidence|mode|mode_confidence|        loudness|start_of_fade_out|time_signature_confidence| artist_hotttnesss|speed|
+--------------------+----------+------------------+--------------+---+--------------+----+---------------+----------------+-----------------+-------------------------+------------------+-----+
|b'SOAAAGO12A67AE0...|        25|1.2004338473567449|           1.0|1.0|         2.168| 1.0|            5.0|4.18689196108551|1.191395662950544|                    2.104|1.8447663111703494|  3.0|
+--------------------+----------+------------------+--------------+---+--------------+----+---------------+----------------+-----------------+-------------------------+------------------+-----+
only showing top 1 row

In [35]:
df_final.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[song_id: string, prediction: bigint, duration: double, end_of_fade_in: double, key: double, key_confidence: double, mode: double, mode_confidence: double, loudness: double, start_of_fade_out: double, time_signature_confidence: double, artist_hotttnesss: double, speed: double]

In [36]:
song_random=df_final.rdd.map(lambda row: row.asDict()).takeSample(False, 1, seed=0) # Choose a random song

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
song_random

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'song_id': "b'SOFTQSW12AB0183155'", 'prediction': 41, 'duration': 1.3516631070730365, 'end_of_fade_in': 1.0007285592654611, 'key': 5.0, 'key_confidence': 3.912, 'mode': 1.0, 'mode_confidence': 5.0, 'loudness': 4.203917050691245, 'start_of_fade_out': 1.3445945611487145, 'time_signature_confidence': 4.779999999999999, 'artist_hotttnesss': 2.314913622364997, 'speed': 3.0}]

In [38]:
from pyspark.sql.functions import col
import time
s = time.time()
prediction_cluster=song_random[0]['prediction']
similar_cluster_rdd=df_final.filter(col("prediction") == prediction_cluster)#All the songs which are assigned to that same cluster
similar_cluster_rdd_new=similar_cluster_rdd.drop(similar_cluster_rdd.prediction).rdd.map(lambda row: row.asDict())
similar_cluster_rdd_new_final= similar_cluster_rdd_new.map(lambda x:[x['song_id'],cossim(x,song_random[0])])   
u = similar_cluster_rdd_new_final.top(5, key=lambda x: x[1])
e = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 1021.0 failed 4 times, most recent failure: Lost task 2.3 in stage 1021.0 (TID 13505, ip-172-31-88-18.ec2.internal, executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1617185507602_0006/container_1617185507602_0006_01_000004/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/mnt/yarn/usercache/livy/appcache/application_1617185507602_0006/container_1617185507602_0006_01_000004/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/mnt/yarn/usercache/livy/appcache/application_1617185507602_0006/container_1617185507602_0006_01_000004/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterat

In [39]:
s-e

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'e' is not defined
Traceback (most recent call last):
NameError: name 'e' is not defined



In [49]:
similar_cluster_rdd.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13928

In [50]:
similar_cluster_rdd_new.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'song_id': "b'SOAQWAL12AB018ABA2'", 'duration': 1.5837063501752038, 'end_of_fade_in': 1.0, 'key': 5.0, 'key_confidence': 3.612, 'mode': 1.0, 'mode_confidence': 5.0, 'loudness': 3.695468509984639, 'start_of_fade_out': 1.5722637526404073, 'time_signature_confidence': 4.132, 'artist_hotttnesss': 2.473472359334762, 'speed': 3.0}]

In [ ]:
import math

def dotprod(a, b):
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    # return <FILL IN>
    
    dot=0
    for k in a :
        if k in b:
            if k!='song_id':
                dot+=a[k]*b[k]  
    return dot

def norm(a):
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    # return <FILL IN>
    
    dot=0
    for k in a:
        if k!='song_id':
            dot+=a[k]*a[k]  
    return math.sqrt(dot)

def cossim(a, b):
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code

    num=dotprod(a, b)
    denom=norm(a)*norm(b)
    # YOUR CODE HERE
    cos_sim=num/denom
    
    return cos_sim

In [52]:
similar_cluster_rdd_new_final.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[["b'SOAQWAL12AB018ABA2'", 0.8459186124657676], ["b'SOBCLMM12AB018DB61'", 0.8477121075767472], ["b'SOBLGQV12AB018EA54'", 0.8463472277215823], ["b'SOCCENB12A8C13595B'", 0.8454402480905036], ["b'SOCDVQP12A8AE465E0'", 0.8471193196193906], ["b'SOCICNU12B0B80BC8D'", 0.8459849181355631], ["b'SODAPPP12A67ADA451'", 0.8474704927811396], ["b'SODJHOK12A8C13FE05'", 0.84627206415591], ["b'SODMTOQ12AB018B3E4'", 0.8479401251262007], ["b'SOEBCVZ12A8C13AC7F'", 0.8471544509787327]]

In [53]:
#https://rsandstroem.github.io/sparkkmeans.html
#The top 5 recommended songs are:
similar_cluster_rdd_new_final.top(5, key=lambda x: x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[["b'SOFTQSW12AB0183155'", 0.84835849800881], ["b'SOKZFRX12AB017EF3A'", 0.8483319044643133], ["b'SOLOJZQ12A8C13CE66'", 0.8483292294166185], ["b'SOUSKHR12A8C137FE6'", 0.8483173320324371], ["b'SOUHMLA12AB0189A5D'", 0.8482781433119553]]